In [ ]:
import os
import datetime
import lxml.etree as et
import csv
pmcdir = 'pmc_articles'
from plos_corpus import (corpusdir, get_article_pubdate, check_if_uncorrected_proof, listdir_nohidden,
                         get_article_xml, file_to_doi, doi_to_file, get_all_solr_dois, download_check_and_move)

from samples.corpus_analysis import (get_plos_article_type, get_article_dtd, get_random_list_of_dois, 
                                     get_related_retraction_article, check_article_type, get_plos_journal,
                                     get_article_title, parse_article_date, get_corpus_metadata,
                                     get_article_abstract, corpus_metadata_to_csv, get_article_dates,
                                     read_corpus_metadata_from_csv)


In [ ]:
article_list = listdir_nohidden(corpusdir)[600:1100]

In [ ]:
corpus_metadata, wrong_dates = get_corpus_metadata(article_list=article_list)

In [5]:
article_list = listdir_nohidden(corpusdir)[60000:60400]
corpus_metadata_to_csv(article_list=article_list)

100% (400 of 400) |################################################################################################################| Elapsed Time: 0:00:15 Time: 0:00:15


In [ ]:
#to execute: update corpus_metadata with a list of DOIs. check that functions can handle an overlapping list, and make
# sure that appending is working correctly. there were some errors reading in a csv that had been appended/extended.

In [ ]:
corpus_metadata_reconstructed = read_corpus_metadata_from_csv()

In [ ]:
corpus_metadata_reconstructed[0]